<a href="https://colab.research.google.com/github/kd303/tensorflow_coursera/blob/master/text_analytics/Course_3_Week_2_Lesson_1-BasicsOfEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0


tf.enable_eager_execution()

# !pip install -q tensorflow-datasets

1.14.0


In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


W0814 05:31:35.049670 140062779963264 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0814 05:31:54.742753 140062779963264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


W0814 05:32:41.168840 140062779963264 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [0]:
print(padded[1])

print(word_index['this'])



[ 360   14    4 1692 1111    3  449   18   13   11 3489  117 7526    5
  150  112  112  207    2  983   14 2026    9 9522   57    2  185   17
    2  933  106   44   25  149    5  178   46  132   36   68   83 1159
   52  156    3   35 1107   66 2763   15   16    1    1   56    7  310
    3    4 3512    6  102  239    5  371    9   65   56  389   11  208
   97    4  210    6   69    1    1    9   31   40  361   56    7    4
  912  530    3    4 1761    6  107   15  257 4376    5   40  110  265
    6  113   11   58   67   29 6092   55   11   69  142   35 1362  436
   16   92    5    2 1284 1275   93   53]
12


In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])



def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b'this was a wonderfully clever and entertaining movie that i shall never tire of watching many many times the casting was magnificent in matching up the young with the older characters there are those of us out here who really do appreciate good actors and an intelligent story format as for <OOV> <OOV> she is beautiful and a gift to any kind of production in which she stars i always make a point to see <OOV> <OOV> in all her performances she is a superb actress and a pleasure to watch as each transformation of her character comes to life i can only be grateful when i see such an outstanding picture for most of the motion pictures made more
b'This was a wonderfully clever and entertaining movie that I shall never tire of watching many, many times. The casting was magnificent in matching up the young with the older characters. There are those of us out here who really do appreciate good actors and an intelligent story format. As for Judi Dench, she is beautiful and a gift to any kind of

'?'

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

W0814 05:53:46.037589 140062779963264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 282us/sample - loss: 0.4863 - acc: 0.7456 - val_loss: 0.3474 - val_acc: 0.8484
Epoch 2/10
25000/25000 [==============================] - 6s 229us/sample - loss: 0.2428 - acc: 0.9057 - val_loss: 0.3769 - val_acc: 0.8353
Epoch 3/10
25000/25000 [==============================] - 6s 228us/sample - loss: 0.1014 - acc: 0.9722 - val_loss: 0.4486 - val_acc: 0.8248
Epoch 4/10
25000/25000 [==============================] - 6s 228us/sample - loss: 0.0273 - acc: 0.9965 - val_loss: 0.5225 - val_acc: 0.8267
Epoch 5/10
25000/25000 [==============================] - 6s 226us/sample - loss: 0.0076 - acc: 0.9992 - val_loss: 0.5858 - val_acc: 0.8254
Epoch 6/10
25000/25000 [==============================] - 6s 228us/sample - loss: 0.0021 - acc: 1.0000 - val_loss: 0.6462 - val_acc: 0.8264
Epoch 7/10
25000/25000 [==============================] - 6s 227us/sample - loss: 9.5928e-04 - acc: 1.0000 - v

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1537], [1537], [4738], [], [790], [2017], [11], [2921], [2190], [], [790], [2017], [11], [579], [], [11], [579], [], [4], [1783], [4], [4506], [11], [2921], [1277], [], [], [2017], [1005], [2921], [968], [579], [790], []]
